In [1]:
from google.colab import drive
path = '/content/drive/'
drive.mount(path)

Mounted at /content/drive/


In [2]:
%cd /content/drive/MyDrive/Custom_Image_Classification/

/content/drive/MyDrive/Custom_Image_Classification


In [3]:
!ls

car_6.jpeg  simple_images


In [4]:
# Extract Features from the dataset

from pathlib import Path
import numpy as np
import joblib
from tensorflow import keras
from tensorflow.keras.preprocessing import image
from keras.applications import convnext

classes = ['car','van','train','bicycle','motorcycle']

paths = []

for c in classes:
    paths.append(Path("simple_images") / c)
    print(paths)

images = []
labels = []

class_index = 0
for path in paths:
    count_image = 0
    for img in path.glob("*.jpeg"):
        img = image.load_img(img,target_size=(224, 224))
        image_array = image.img_to_array(img)
        images.append(image_array)
        labels.append(class_index)
        count_image += 1
    print('The number of ', classes[class_index], count_image)
    class_index += 1

x_train = np.array(images)
y_train = np.array(labels)

# Normalize image data to 0-to-1 range
x_train = convnext.preprocess_input(x_train)

pretrained_nn = convnext.ConvNeXtBase(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
features_x = pretrained_nn.predict(x_train)
joblib.dump(features_x, "x_train.dat")
joblib.dump(y_train, "y_train.dat")

[PosixPath('simple_images/car')]
[PosixPath('simple_images/car'), PosixPath('simple_images/van')]
[PosixPath('simple_images/car'), PosixPath('simple_images/van'), PosixPath('simple_images/train')]
[PosixPath('simple_images/car'), PosixPath('simple_images/van'), PosixPath('simple_images/train'), PosixPath('simple_images/bicycle')]
[PosixPath('simple_images/car'), PosixPath('simple_images/van'), PosixPath('simple_images/train'), PosixPath('simple_images/bicycle'), PosixPath('simple_images/motorcycle')]
The number of  car 80
The number of  van 79
The number of  train 92
The number of  bicycle 83
The number of  motorcycle 85
350926856/350926856 ━━━━━━━━━━━━━━━━━━━━ 9s 0us/step
14/14 ━━━━━━━━━━━━━━━━━━━━ 21s 807ms/step


['y_train.dat']

In [5]:
# Training a model

from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from pathlib import Path
import joblib

# Load data set
x_train = joblib.load("x_train.dat")
y_train = joblib.load("y_train.dat")

# Create a model and add layers
model = Sequential()

model.add(Flatten(input_shape=x_train.shape[1:]))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(5, activation='softmax'))  # Output Layers # Numbers can change here

# Compile the model
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer="adam",
    metrics=['accuracy']
)

# Train the model
model.fit(
    x_train,
    y_train,
    epochs=100,
    shuffle=True
)

# Save neural network structure
model_structure = model.to_json()
f = Path("model_structure.json")
f.write_text(model_structure)

# Save neural network's trained weights
model.save_weights("model.weights.h5")


/usr/local/lib/python3.11/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 3s 83ms/step - accuracy: 0.7723 - loss: 1.6016
Epoch 2/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.9900 - loss: 0.1191
Epoch 3/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.9931 - loss: 0.0235
Epoch 4/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.9969 - loss: 0.0294
Epoch 5/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.9856 - loss: 0.0807
Epoch 6/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.9860 - loss: 0.0561
Epoch 7/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.9963 - loss: 0.0975
Epoch 8/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.9972 - loss: 0.0610
Epoch 9/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 1.0000 - loss: 3.3262e-06
Epoch 10/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 1.0000 - loss: 5.2653e-07
Epoch 11/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 1.0000 - loss: 0.0000e+00
Epoch 12/100
14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms

In [12]:
# Predicting the image using the trained model

from keras.models import model_from_json
from pathlib import Path
from tensorflow.keras.preprocessing import image
import numpy as np
from keras.applications import convnext

# Load the json file that contains the model's structure
f = Path("model_structure.json")
model_structure = f.read_text()

# Recreate the Keras model object from the json data
model = model_from_json(model_structure)

# Re-load the model's trained weights
model.load_weights("model.weights.h5")

# Load an image file to test, resizing it to 224x224 pixels (as required by this model)
img = image.load_img("car_6.jpeg", target_size=(224, 224))

# Convert the image to a numpy array
image_array = image.img_to_array(img)

# Add a forth dimension to the image (since Keras expects a bunch of images, not a single image)
images = np.expand_dims(image_array, axis=0)

# Normalize the data
images = convnext.preprocess_input(images)

# Use the pre-trained neural network to extract features from our test image (the same way we did to train the model)
feature_extraction_model = convnext.ConvNeXtBase(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
features = feature_extraction_model.predict(images)

# Given the extracted features, make a final prediction using our own model
results = model.predict(features)
print('Probability:', results)

predicted_class = np.argmax(results)
predicted_name = 'None'
if predicted_class==0:
    predicted_name = 'Car'
elif predicted_class==1:
    predicted_name = 'Van'
elif predicted_class==2:
    predicted_name = 'Train'
elif predicted_class==3:
    predicted_name = 'Bicycle'
else:
    predicted_name = 'Motorcycle'
print('This is',predicted_name,'with confidence:',results[0][np.argmax(results)]*100)

1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 411ms/step
Probability: [[1. 0. 0. 0. 0.]]
This is Car with confidence: 100.0
